In [ ]:
import json
import os
import numpy
import pandas
import sys
import logging
import importlib

from datetime import date
from os import path
from dotenv import load_dotenv

sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

from bespoke.inventory.analysis.shared import download_util
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util
from bespoke.inventory.analysis import inventory_cogs_util as cogs_util
from bespoke.inventory.analysis import inventory_summary_util
from bespoke.inventory.analysis import stale_inventory_util
from bespoke.inventory.analysis.shared.inventory_types import (
  AnalysisSummaryDict,
  FacilityDetailsDict,
  CompanyInfoDict
)

load_dotenv(verbose=True)
engine = download_util.get_bigquery_engine('bigquery://bespoke-financial/ProdMetrcData')

logging.basicConfig(format='%(asctime)s [%(levelname)s] - %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

def _reload_libs():
    importlib.reload(util)
    importlib.reload(valuations_util)
    importlib.reload(download_util)
    importlib.reload(stale_inventory_util)

%load_ext autoreload
%autoreload 2

In [ ]:
COMPANY_NAME = 'RA'
COMPANY_IDENTIFIER = 'RA'
LICENSE_NUMBERS = []
FACILITY_DETAILS = FacilityDetailsDict(
    license_numbers=LICENSE_NUMBERS,
    name='default'
)
TRANSFER_PACKAGES_START_DATE = '2019-01-01'
SALES_TRANSACTIONS_START_DATE = '2019-01-01'
ANALYSIS_PARAMS = {
    'sold_threshold': 1.0,
    'find_parent_child_relationships': False,
    'use_prices_to_fill_missing_incoming': False,
    'external_pricing_data_config': {
        'category_to_fixed_prices': {
            'Buds': {
                'grams': 10.0
            },
            'Infused (edible)': {
                'each': 4.0
            },
            'Infused (non-edible)': {
                'each': 3.0
            },
            'Vape Product': {
                'each': 3.0
            },
            'Concentrate (Bulk)': {
                'grams': 6.0
            },
            'Concentrate': {
                'grams': 7.0
            },
            'Raw Pre-Rolls': {
                'grams': 7.0,
                'pounds': 80.0
            },
            'Shake/Trim (by strain)': {
                'grams': 8.0
            }
        }
    },
    'use_margin_estimate_config': False,
    'margin_estimate_config': {
        'category_to_margin_estimate': {
            'Buds': 0.4267,
            'Infused (edible)': 0.4953,
            'Infused (non-edible)': 0.4183,
            'Vape Product': 0.43,
            'Concentrate (Bulk)': 0.46,
            'Concentrate': 0.46,
            'Raw Pre-Rolls': 0.485,
            'Shake/Trim (by strain)': 0.4267
        }
    },
    'cogs_analysis_params': {
        'readjust_profit_threshold': 0.90, # 0.9
        'readjust_type': 'adjust' # adjust, remove
    },
    'stale_inventory_params': {
        'product_category_to_shelf_life': {
            "Flower": 6,
            "Trim": 6,
            "Fresh Frozen": 0,
            "Edibles": 6,
            "Wax": 12,
            "Resin": 12,
            "Tinctures": 12,
            "Vapes": 12,
            "Shatter": 12,
            "Concentrates": 12,
            "Rosin": 12,
            "Beverages": None,
            "unknown": 12           
        }
    }
}
TODAY_DATE = date.today()
print('Today is {}'.format(TODAY_DATE))

download_ctx = download_util.DataframeDownloadContext(
    output_root_dir=f'out/{COMPANY_NAME}',
    read_params={
        'use_cached_dataframes': False
    },
    write_params={
        'save_download_dataframes': False
    }
)
download_ctx.mkdir('download')

ctx = download_util.AnalysisContext(
    output_root_dir=f'out/{COMPANY_NAME}'
)
ctx.mkdir('reports')
with open(ctx.get_output_path('log.txt'), 'w') as f:
    f.write('')

In [ ]:
# Download packages, sales transactions, incoming / outgoing tranfers
df_query_params = download_util.DataFrameQueryParams(
    company_identifier=COMPANY_IDENTIFIER,
    transfer_packages_start_date=TRANSFER_PACKAGES_START_DATE,
    sales_transactions_start_date=SALES_TRANSACTIONS_START_DATE,
    license_numbers=LICENSE_NUMBERS
)
sql_helper = download_util.BigQuerySQLHelper(download_ctx, engine)
all_dataframes_dict = download_util.get_dataframes_for_analysis(
    df_query_params, download_ctx, sql_helper, dry_run=False, num_threads=2, use_incremental_querying=False)

In [ ]:
q = download_util.Query(
    inventory_dates=download_util.get_inventory_dates(
      all_dataframes_dict, TODAY_DATE),
    company_id='',
    company_name=COMPANY_NAME,
    company_identifier=COMPANY_IDENTIFIER,
)
d = util.Download()
d.process_dataframes(
    all_dataframes_dict=all_dataframes_dict,
    ctx=ctx
)

In [ ]:
id_to_history = util.get_histories(d, ANALYSIS_PARAMS)
util.print_counts(ctx, id_to_history)
compute_inventory_dict = util.create_inventory_xlsx(
    d, ctx, id_to_history, q, params=ANALYSIS_PARAMS, using_nb=True)

In [ ]:
today_date_str = TODAY_DATE.strftime('%m/%d/%Y')
compare_inventory_res = util.compare_computed_vs_actual_inventory(
    ctx=ctx,
    computed=compute_inventory_dict['date_to_computed_inventory_dataframe'][today_date_str],
    actual=d.inventory_packages_dataframe,
    params=ANALYSIS_PARAMS,
    compare_options={
        'num_errors_to_show': 10,
        'accept_computed_when_sold_out': True,
    },
    today=TODAY_DATE
)


In [ ]:
## Inventory valuations
date_and_computed_valuation = list(zip(q.inventory_dates, compute_inventory_dict['inventory_valuations']))[-1]

print('Cost valuation based on computed inventory as of {} is ${}'.format(
  date_and_computed_valuation[0], round(date_and_computed_valuation[1], 2)
))

inventory_cost_valuation_dict = valuations_util.get_inventory_valuation(
    inventory_packages_dataframe=d.inventory_packages_dataframe,
    incoming_transfer_packages_dataframe=d.incoming_transfer_packages_dataframe,
    params=ANALYSIS_PARAMS,
    today=TODAY_DATE
)

metrc_cost_total_valuation = inventory_cost_valuation_dict['total_valuation']
metrc_cost_fresh_valuation = inventory_cost_valuation_dict['total_fresh_valuation']

print(f'Cost valuation of Metrc-reported total inventory as of today: ${round(metrc_cost_total_valuation, 2)}')
print(f'Cost valuation of Metrc-reported fresh inventory as of today: ${round(metrc_cost_fresh_valuation, 2)}')

In [ ]:
print(f'Plotting sales revenue vs cost-based inventory valuation for dates: {q.inventory_dates}')
valuations_util.plot_inventory_and_revenue(
    q=q,
    sales_receipts_dataframe=d.sales_receipts_dataframe,
    inventory_valuations=compute_inventory_dict['inventory_valuations'],
    fresh_inventory_valuations=compute_inventory_dict['fresh_inventory_valuations']
)